# Activity 4: Creating failover groups using PowerShell  

#### <i>The Azure SQL Workshop - Module 5</i>

<p style="border-bottom: 1px solid lightgrey;"></p>

In the module, you learned about geo-replication and auto-failover groups. In this activity, you'll configure auto-failover groups for your Azure SQL Database. You'll then initiate a failover and observe the results, leveraging an application.  


**Set up**   

0. You should have opened this file using Azure Data Studio. If you didn't, please refer to Module 2 Activity 3 in the readme.md file to get set up.  
1. In the bar at the top of this screen, confirm or change the "Kernel" to **PowerShell**. This determines what language the code blocks in the file are. In this case, that language is PowerShell.  
2. Confirming the Kernel is **PowerShell**, for "Attach to", it should read **localhost**.  

Now that you're set up, you should read the text cells and "Run" the code cells by selecting the play button that appears in the left of a code cell when you hover over it.  


## Auto-failover groups in Azure SQL  

To configure auto-failover groups for a database(s) and observe the results in an application, you must complete the following steps:  

0. Confirm environment is properly configured
1. Create an empty Azure SQL Database server in the failover region
1. Create a failover group between the servers  
1. Add database(s) to the failover group  
1. Configure the network
1. Configure your application(s)
1. Run your application(s)
1. Initiate a failover
1. Fail back
1. (Bonus) How many read replicas?  

This notebook will guide you through configuring auto-failover groups for your AdventureWorksID database. You'll then use a simple Java application to understand where reads and writes occur, and the importance of retry logic in your applications. Finally, you'll do a fun exercise to determine how many read-replicas are associated with a Business critical database that also has an auto-failover group.  

>Note: This activity is based off of a [tutorial in the documentation](https://docs.microsoft.com/en-us/azure/sql-database/sql-database-implement-geo-distributed-database?tabs=azure-powershell) that also has information about using the Azure portal and the Azure CLI. In this exercise, you will use the Az PowerShell module.  

**Step 0 - Confirm environment is properly configured**

Confirm the [Az PowerShell Module](https://docs.microsoft.com/en-us/powershell/azure/install-az-ps?view=azps-3.4.00) is installed with version 3.4.0+. 

In [1]:
Get-InstalledModule -Name Az -AllVersions | select Name,Version


Name Version
---- -------
Az   3.5.0  




Confirm [Maven and Java](https://www.microsoft.com/en-us/sql-server/developer-get-started/java/windows) are installed and configured with Maven 3.6.3+ and Java 1.8.0_241+. 

In [2]:
mvn --version

Apache Maven 3.6.3 (cecedd343002696d0abb50b32b541b8a6ba2883f)
Maven home: C:\Users\vmuser\Downloads\apache-maven-3.6.3-bin\apache-maven-3.6.3\bin\..
Java version: 1.8.0_241, vendor: Oracle Corporation, runtime: C:\Program Files\Java\jdk1.8.0_241\jre
Default locale: en_US, platform encoding: Cp1252
OS name: "windows 10", version: "10.0", arch: "amd64", family: "windows"


If either of the above cells fail, please revisit the prerequisites before continuing.  

In the below cell, replace the `0406` with the ID you are using for the workshop. Also, add your subscription ID, your SQL authentication password, and the location of your main Azure SQL Database logical server. For `$drLocation`, you'll need to supply the region where you want your failover group to be. Ideally, you would select a paired region, which you can see the list of [here](https://docs.microsoft.com/en-us/azure/best-practices-availability-paired-regions). 

> Note: If you're taking this as part of an in-person workshop and were provided an environment to use, defer to instructor guidance on what region to use.  

In [ ]:
# Add your info
$unique_id = "0406"
$subscriptionId = "subscription"
$location = "location"
$password = "db password"
$drLocation = "dr location"

# Do not modify
$admin = "cloudadmin"
$resourceGroup = "azuresqlworkshop$($unique_id)"
$server = "aw-server$($unique_id)"
$database = "AdventureWorks$($unique_id)"
$drServer = "aw-server$($unique_id)-dr"
$failoverGroup = "aw-server-fg-$($unique_id)"
$firewallRule = "VmFirewall"
$ip = Invoke-RestMethod -Uri 'https://ipinfo.io/ip'
$ipAddress = $ip.Substring(0,$ip.Length-1)  
Write-Host "Variables Received"

In [ ]:
Set-AzContext -SubscriptionId $subscriptionId 

**Step 1 - Create an empty Azure SQL Database server in the failover region**

In [ ]:
# create a backup server in the failover region
New-AzSqlServer -ResourceGroupName $resourceGroup `
    -ServerName $drServer `
    -Location $drLocation `
    -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential `
    -ArgumentList $admin, $(ConvertTo-SecureString -String $password -AsPlainText -Force))
Write-Host "New Azure SQL Database logical server Created in different region"

**Step 2 - Create a failover group between the servers**

In [ ]:
# create a failover group between the servers
New-AzSqlDatabaseFailoverGroup -ResourceGroupName $resourceGroup `
    -ServerName $server `
    -PartnerServerName $drServer `
    -FailoverGroupName $failoverGroup 
Write-Host "New auto-failover group created between the two Azure SQL Database logical servers"

**Step 3 - Add database(s) to the failover group**

In [ ]:
# add the database to the failover group
Get-AzSqlDatabase -ResourceGroupName $resourceGroup `
    -ServerName $server -DatabaseName $database | `
    Add-AzSqlDatabaseToFailoverGroup -ResourceGroupName $resourceGroup `
    -ServerName $server `
    -FailoverGroupName $failoverGroup
Write-Host "AdventureWorks database added to the auto-failover group"

The above step takes some time, because what you're doing is restoring the database in the other region, which involves copying the data from the original region to the DR region. You can work on **Step 5** and then come back here to check if it has completed. If it hasn't, you can work on **Step 6** and then check back here.  

**Step 4 - Configure the network**

In [ ]:
# Add a firewall rules that gives your VM access to the new server
New-AzSqlServerFirewallRule -ResourceGroupName $resourceGroup `
    -ServerName $drServer `
    -FirewallRuleName $firewallRule `
    -StartIpAddress $ipAddress `
    -EndIpAddress $ipAddress;

What you've just done is deploy and configure and auto-failover group for you AdventureWorks database. Let's review.  

You started with a database and server in one location, with access from a virtual network (containing your VM) in that same location, this is what you did in Modules 2 and 3. In this notebook, you created a failover group from your original server to a new server in a different location, and added your AdventureWorks database to it. Finally, you added a firewall rule so that the new server/database in a different region can still be accessed from your VM.  

For the purposes of this activity (illustrating auto-failover groups) this is sufficient from a networking and VM perspective. However, this is slightly different from what you would do in an enterprise environment. In an enterprise environment, your "VM" will likely be a set of resources that make up some type of application. In the event of a failover of your database, you may want to failover your application/VMs/other resources to that new region as well. Both sets of resources will need access to the resources/servers/databases in the other region. In order to do this, you can use virtual network peering, virtual network to virtual network connections, or potentially something else (like ExpressRoute). It will depend on your scenario. You can learn more about what is needed to [design globally available services using Azure SQL Database here](https://docs.microsoft.com/en-us/azure/sql-database/sql-database-designing-cloud-solutions-for-disaster-recovery).  

**Step 5 - Configure your application(s)**  

For this activity, you'll leverage a sample Java application that attempts to write on your primary server and read what was just written from your secondary server. In order to use this app, you need to add your information to four lines of code. From the **Explorer** left-hand menu in Azure Data Studio, navigate to `azuresqlworkshop\05-Availability\fg\app\src\main\java\com\sqldbsamples\App.java`. Clicking on the `App.java` file should open a text editor within Azure Data Studio. Update the lines near the top that require your specific information and then save the file. You only need to update the following lines (snippet from `App.java`):  

```java
   // UPDATE WITH YOUR INFO
   private static final String FAILOVER_GROUP_NAME = "aw-server-fg-ID";  // add workshop ID
   // UPDATE WITH YOUR INFO
   private static final String DB_NAME = "AdventureWorksID";  // add workshop ID
   // UPDATE WITH YOUR INFO
   private static final String USER = "cloudadmin";  // add database user
   // UPDATE WITH YOUR INFO
   private static final String PASSWORD = "<Password>";  // add database password
```  

Review this file and determine what the two queries that are running do.  

Be sure to save the file before you close it.  

**Step 6 - Run your application(s)**

This app runs from the command-line. Open Command prompt (you can type `cmd` in the Windows search bar and select **Enter**).  

If you've followed the prerequisites, you just need to update `<vmuser>` to the VM username you use to log in to this VM. Update and then run the following commands in Command prompt.  

```cmd
cd C:\Users\vmuser\sqlworkshops\AzureSQLWorkshop\azuresqlworkshop\05-Availability\fg\app
mvn package
mvn -q -e exec:java "-Dexec.mainClass=com.sqldbsamples.App"
```

The first 5-10 inserts may fail. As long as it settles to both being successful, your app is running properly.  

For the next steps, it is recommended to split your screen so you can see both Azure Data Studio and that command prompt window. Example below:  

![](./graphics/splitscreen.png)


**Step 7 - Initiate a failover**   


Check the status of the secondary server.

In [9]:
(Get-AzSqlDatabaseFailoverGroup -FailoverGroupName $failoverGroup `
    -ResourceGroupName $resourceGroup -ServerName $drServer).ReplicationRole

Secondary


The result of the previous cell tells you if the secondary server in the auto-failover group is being used as the primary or secondary database.

We can now see what happens when a failover occurs by starting a manual failover with the Azure PowerShell commands below.  

When the failover occurs, you'll notice that some of the inserts and reads may fail briefly. But since our app keeps retrying, the application doesn't fail. And once the failover has completed, the inserts and reads become successful again.  

In [10]:
Switch-AzSqlDatabaseFailoverGroup -ResourceGroupName $resourceGroup `
 -ServerName $drServer -FailoverGroupName $failoverGroup

Check the status of the secondary server.

In [11]:
(Get-AzSqlDatabaseFailoverGroup -FailoverGroupName $failoverGroup `
    -ResourceGroupName $resourceGroup -ServerName $drServer).ReplicationRole

Primary


**Step 8 - Fail back**  

Now we can fail back to our original configuration by using a similar command.  

In [12]:
Switch-AzSqlDatabaseFailoverGroup -ResourceGroupName $resourceGroup `
 -ServerName $server -FailoverGroupName $failoverGroup

Check the status of the secondary server.

In [13]:
(Get-AzSqlDatabaseFailoverGroup -FailoverGroupName $failoverGroup `
    -ResourceGroupName $resourceGroup -ServerName $drServer).ReplicationRole

Secondary


**(Bonus) Step 9 - How many read replicas?**

You've seen how you can use the primary/secondary in an auto-failover group to help offload read-heavy workloads from the primary. In the previous activity, you also saw how in certain service tiers, like Business critical, you get a read-replica without setting up auto-failover groups. This begs the question, how many read-replicas do you have with a Business critical Azure SQL Database with an auto-failover group?  

Maybe you guessed it, but the answer is **3**. You can investigate this claim by using SSMS (or ADS) to connect to the following servers (be sure to specify the database under connection properties) and running the query to check if it's ReadWrite or ReadOnly below.  

The easiest way to do this is to create a New Database Engine Query by selecting **File** > **New** > **Database Engine Query**.  

![](./graphics/newdbenginequery.png)

Connections:  
* `aw-server-fg-`YourID`.database.windows.net`  
* `aw-server-fg-`YourID`.database.windows.net` with Additional Connection Parameter `ApplicationIntent=ReadOnly`  
* `aw-server-fg-`YourID`.secondary.database.windows.net`  
* `aw-server-fg-`YourID`.secondary.database.windows.net` with Additional Connection Parameter `ApplicationIntent=ReadOnly`  

Command to run in each:  
```sql  
SELECT DATABASEPROPERTYEX(DB_NAME(), 'Updateability')
```  